In [ ]:
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

In [ ]:
geolocator = Nominatim(user_agent="UrbanTopologyAnalysisService")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [ ]:
data = pd.read_csv("../../data/legal_entities.csv", index_col=0)
data

In [ ]:
data["Количество торговых объектов местного значения, ед."] = data["Количество торговых объектов местного значения, ед."].apply(lambda x: int(x.replace(" ", "")) if x != "н/д" else None)
data

In [ ]:
data_not_null = data.loc[~data["Количество торговых объектов местного значения, ед."].isnull()]
data_not_null

In [ ]:
clear_data = data_not_null.drop(data_not_null[data_not_null["№ п/п"].str.len() <= 3].index)
clear_data

In [ ]:
clear_data.loc[clear_data["Наименование внутригородского муниципального образования"].str.startswith("поселок"), "Наименование внутригородского муниципального образования"] = clear_data.loc[clear_data["Наименование внутригородского муниципального образования"].str.startswith("поселок"), "Наименование внутригородского муниципального образования"].apply(lambda x: x.replace("поселок ", ""))
clear_data.loc[clear_data["Наименование внутригородского муниципального образования"].str.startswith("поселок")]

In [ ]:
def apply_geocode(area):
    location = geocode(area + ", Санкт-Петербург")
    return (location.latitude, location.longitude) if location else None

clear_data["Геопозиция"] = clear_data["Наименование внутригородского муниципального образования"].progress_apply(apply_geocode)

In [ ]:
clear_data.loc[clear_data["Геопозиция"].isnull()]

In [ ]:
clear_data.to_csv("legal_entities.csv")